<a href="https://colab.research.google.com/github/Suhas-S63/Deep_Learning_Lab/blob/main/563_DL_Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### YOLO with Custom Data Dataset

In [ ]:
from ultralytics import YOLO
import cv2
import math

In [4]:
import os
count = 0
# paths
image_directory = 'D://Datasets//vehicles'
annot_directory = 'D://Datasets//labels_vehicles_notes'

# Iterate through each image file in the image folder
for filename in os.listdir(image_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = os.path.join(image_directory, filename)
        annotation_path = os.path.join(annot_directory, os.path.splitext(filename)[0] + '.txt')

        # Check if corresponding annotation file exists
        if os.path.exists(annotation_path):
            # Process the image and annotation
            print(f"Image: {image_path}, Annotation: {annotation_path}")
            count+=1
        else:
            print(f"No annotation found for {filename}")
print("Total images with annotations: " + str(count))

Image: D://Datasets//vehicles\auto.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto.txt
Image: D://Datasets//vehicles\auto10.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto10.txt
Image: D://Datasets//vehicles\auto11.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto11.txt
Image: D://Datasets//vehicles\auto12.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto12.txt
Image: D://Datasets//vehicles\auto13.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto13.txt
Image: D://Datasets//vehicles\auto14.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto14.txt
Image: D://Datasets//vehicles\auto15.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto15.txt
Image: D://Datasets//vehicles\auto16.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto16.txt
Image: D://Datasets//vehicles\auto17.jpeg, Annotation: D://Datasets//labels_vehicles_notes\auto17.txt
Image: D://Datasets//vehicles\auto18.jpeg, Annotation: D://Datasets//labels_vehicles_n

In [7]:
import os
import cv2

# paths
image_directory = 'D://Datasets//vehicles'
annot_directory = 'D://Datasets//labels_vehicles_notes'

# storing preprocessed data in the new directory
preprocessed_image_directory = 'D://Datasets//Preprocessed_Images'
preprocessed_annot_directory = 'D://Datasets//Preprocessed_Annotations'
os.makedirs(preprocessed_image_directory, exist_ok=True)
os.makedirs(preprocessed_annot_directory, exist_ok=True)


for filename in os.listdir(image_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = os.path.join(image_directory, filename)
        annotation_path = os.path.join(annot_directory, os.path.splitext(filename)[0] + '.txt')
        # check annotations
        if os.path.exists(annotation_path):
            # Image
            image = cv2.imread(image_path)
            # read annot file
            with open(annotation_path, 'r') as file:
                for line in file:
                    # Parsing the annotations in the YOLO format
                    class_id, x_center, y_center, width, height = map(float, line.split())
                    # Convert relative coordinates to absolute coordinates
                    img_height, img_width, _ = image.shape
                    x_center_abs = int(x_center * img_width)
                    y_center_abs = int(y_center * img_height)
                    bbox_width = int(width * img_width)
                    bbox_height = int(height * img_height)
                    # top-left corner coordinates calc
                    x1 = int(x_center_abs - bbox_width / 2)
                    y1 = int(y_center_abs - bbox_height / 2)
                    #  resize the image to a fixed size
                    resized_image = cv2.resize(image, (224, 224))
                    cv2.imwrite(os.path.join('D://Datasets//Preprocessed_Images', filename), resized_image)
                    # Save the annotation file as it is
                    with open(os.path.join('D://Datasets///Preprocessed_Annotations', os.path.splitext(filename)[0] + '.txt'), 'w') as annot_file:
                        annot_file.write(line)
        else:
            print(f"No annotation found for {filename}")


AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

prep_image_directory = 'D://Datasets//Preprocessed_Images'
prep_annotation_directory = 'D://Datasets///Preprocessed_Annotations'

#  data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

#  lists to store image and annotation paths
images = []
annotations = []

for filename in os.listdir(prep_image_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = os.path.join(prep_image_directory, filename)
        annotation_path = os.path.join(prep_annotation_directory, os.path.splitext(filename)[0] + '.txt')

        # Check if annotation file exists
        if os.path.exists(annotation_path):
            images.append(image_path)
            annotations.append(annotation_path)

train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)

# Function to perform data augmentation and save images
def augment_and_save_images(image_paths, annotation_paths, target_directory):
    for image_path, annotation_path in zip(image_paths, annotation_paths):
        image_name = os.path.basename(image_path)
        annotation_name = os.path.basename(annotation_path)
        image = cv2.imread(image_path)
        image = np.expand_dims(image, axis=0)
        it = datagen.flow(image, batch_size=1)
        for i in range(9):
            augmented_image = it.next()[0].astype('uint8')
            augmented_image_name = os.path.splitext(image_name)[0] + f'_augmented_{i+1}.jpg'
            augmented_image_path = os.path.join(target_directory, augmented_image_name)
            cv2.imwrite(augmented_image_path, augmented_image)

# Augment and save images for training set
augment_and_save_images(train_images, train_annotations, prep_image_directory)
print("Augmented images generated for training set.")

# Augment and save images for validation set
augment_and_save_images(val_images, val_annotations, prep_image_directory)
print("Augmented images generated for validation set.")

# Augment and save images for test set
augment_and_save_images(test_images, test_annotations, prep_image_directory)
print("Augmented images generated for test set.")


Augmented images generated for training set.
Augmented images generated for validation set.
